This Jupyter notebook file will be used to interview people and find the audio specifications needed for our project.

In [1]:
import random
from scipy.io.wavfile import read
import sounddevice as sd
import numpy as np
import soundfile as sf
import time
import pyttsx3
from pydub import AudioSegment
import pydub.playback as pb

Test 1: Audio direction

For this test, participants will hear 10 sounds, each coming from one of five directions. The directions are classified as W, NW, N, NE, and E. The participant will be asked to determine where the sound came from. At the end, an accuracy score and the individual results will be given.

In [2]:
def convolve_and_play_audio(direction, input_audio, freq):
    L_hrtf_data, freq2 = sf.read('hrtf_sound_files/L_' + direction + '.wav')
    R_hrtf_data, freq2 = sf.read('hrtf_sound_files/R_' + direction + '.wav')
    left_audio = np.convolve(input_audio, L_hrtf_data)
    right_audio = np.convolve(input_audio, R_hrtf_data)
    output_audio = np.vstack((left_audio, right_audio)).T
    sd.play(output_audio[:88200], freq)
    sd.wait()

In [3]:
# Initialize variables needed for experiment
directions = ['W', 'NW', 'N', 'NE', 'E']
degrees_directions = [0, 45, 90, 135, 180]
trials = 10
input_audio_file = 'base_audio.wav'
input_audio, freq = sf.read(input_audio_file)
speech = pyttsx3.init()

# Calibration stage - play one second of each direction; going West to East
speech.say("Calibration. Get familiar with the sounds.")
speech.runAndWait()
for direction in directions:
    convolve_and_play_audio(direction, input_audio, freq)
time.sleep(2)
for direction in reversed(directions):
    convolve_and_play_audio(direction, input_audio, freq)

# Test: Play audio and record results
time.sleep(3)
speech.say("Test. Audio will be played. Record your results.")
speech.runAndWait()
real_directions = []
user_picked_directions = []
correct = 0
incorrect = 0
degrees_incorrect = 0
for trial in range(trials):
    direction = random.choice(directions)
    convolve_and_play_audio(direction, input_audio, freq)
    real_directions.append(direction)
    user_input = input('What direction did the sound come from? (W, NW, N, NE, E) ')
    user_picked_directions.append(user_input)
    if real_directions[-1] == user_picked_directions[-1]:
        correct += 1
    else:
        degrees_incorrect += np.abs(degrees_directions[list(directions).index(direction)] - degrees_directions[list(directions).index(user_input)])
        incorrect += 1
    

print('Real Directions: ', real_directions)
print('User Picked Directions: ', user_picked_directions)
print('Correct: ', correct)
print('Average degree separation for incorrect: ', degrees_incorrect/incorrect)



KeyboardInterrupt: 

Test 2: Volume

For this test, participants will first find the minimum sound that they can hear. Then, they will identify noticeable changes from there. The maximum volume will be the original input audio 
(1 * input_audio)

In [4]:
# Get input audio
input_audio_file = 'base_audio.wav'
input_audio, freq = sf.read(input_audio_file)

# Find minimum volume
userInput = 'N'
factor = 0.001
increase = 0.001
speech.say("Test. Audio will be played. Indicate if you heard the sound.")
speech.runAndWait()
while userInput == 'N':
    output_audio = factor * input_audio
    sd.play(output_audio)
    sd.wait()
    userInput = input('Could you hear the sound? (Y/N) ')
    if userInput == 'N':
        factor += increase

print('Minimum volume heard: ', factor)

# Find noticeable changes
# Low volume
difference = 0.01
userInput = 'N'
time.sleep(2)
speech.say("Test. Audio will be played. Indicate if you heard the difference in volume between the two audios.")
speech.runAndWait()
while userInput == 'N':
    output_audio_1 = input_audio * 0.01
    output_audio_2 = input_audio * (0.01 + difference)
    sd.play(output_audio_1)
    sd.wait()
    time.sleep(1)
    sd.play(output_audio_2)
    sd.wait()
    userInput = input("Could you hear the volume difference? (Y/N) ")
    if userInput == 'N':
        difference += 0.01
print('Noticeable difference factor (low volume): ', difference)

# Medium volume
difference = 0.05
userInput = 'N'
time.sleep(2)
speech.say("Test. Audio will be played. Indicate if you heard the difference in volume between the two audios.")
speech.runAndWait()
while userInput == 'N':
    output_audio_1 = input_audio * 0.4
    output_audio_2 = input_audio * (0.4 - difference)
    sd.play(output_audio_1)
    sd.wait()
    time.sleep(1)
    sd.play(output_audio_2)
    sd.wait()
    userInput = input("Could you hear the volume difference? (Y/N) ")
    if userInput == 'N':
        difference += 0.05
print('Noticeable difference factor (medium volume): ', difference)

# High volume
difference = 0.05
userInput = 'N'
time.sleep(2)
speech.say("Test. Audio will be played. Indicate if you heard the difference in volume between the two audios.")
speech.runAndWait()
while userInput == 'N':
    output_audio_1 = input_audio * 0.7
    output_audio_2 = input_audio * (0.7 + difference)
    sd.play(output_audio_1)
    sd.wait()
    time.sleep(1)
    sd.play(output_audio_2)
    sd.wait()
    userInput = input("Could you hear the volume difference? (Y/N) ")
    if userInput == 'N':
        difference += 0.05
print('Noticeable difference factor (high volume): ', difference)


Minimum volume heard:  0.001
Noticeable difference factor (low volume):  0.01
Noticeable difference factor (medium volume):  -0.15000000000000002
Noticeable difference factor (high volume):  0.05


Test 3: Emotions

For this test, participants will rank audios from most calm to most danger. Three audios will be presented.

In [5]:
# Get input audio
input_audio_file = 'base_audio.wav'
input_audio, freq = sf.read(input_audio_file)

output_audio_1 = input_audio[:132300]  # 3 seconds
output_audio_2 = input_audio[:33075]  # 0.75 seconds
output_audio_3 = input_audio[:11025]  # 0.25 seconds

# Output all 3 audio sequences
speech.say("Test. Three audios will be played. Rank them from most to least calming.")
speech.runAndWait()
time.sleep(1)
# Constant sound
speech.say("Audio 1.")
speech.runAndWait()
time.sleep(1)
sd.play(output_audio_1)
sd.wait()
time.sleep(2)

# Slow beeping
speech.say("Audio 2.")
speech.runAndWait()
time.sleep(1)
for i in range(3):
    sd.play(output_audio_2)
    sd.wait()
    time.sleep(0.25)
time.sleep(2)

# Fast beeping
speech.say("Audio 3.")
speech.runAndWait()
time.sleep(1)
for i in range(6):
    sd.play(output_audio_3)
    sd.wait()
    time.sleep(0.25)
time.sleep(2)

userInput = input('Rank the sounds (eg. 2 1 3): ')

print(userInput)

1 2 3
